In [1]:
import spynnaker8 as p
from pyNN.random import NumpyRNG, RandomDistribution
from pyNN.utility.plotting import Figure, Panel
import matplotlib.pyplot as plt

import time
import numpy as np
import math
import cv2
import pdb


import sys
sys.path.insert(1, '../../miscelaneous')
from spinnorse_tools import fetch_files, plot_in_v_out, plot_spikes, plot_voltages, plot_ma_from_spikes, plot_trajectories
from stimulator import produce_data


In [2]:

def send_spikes_to_sim(label, connection):
    if label == "one":
        rate = 0
    if label == "two":        
        rate = 0
    if label == "three":       
        rate = 10        
    if label == "four":       
        rate = 10
        
    connection.set_rates(label, [(i,  rate) for i in range(1)])
        
def receive_spikes_from_sim(label, time, neuron_ids):
    print("1 for " + label)
    

In [3]:
dt = 1           # (ms) simulation timestep

cell_params = {'tau_m': 20.0,
               'tau_syn_E': 5.0,
               'tau_syn_I': 5.0,
               'v_rest': -65.0,
               'v_reset': -65.0,
               'v_thresh': -50.0,
               'tau_refrac': 0.0, # 0.1 originally
               'cm': 1,
               'i_offset': 0.0
               }

w = 4

In [4]:
nb_steps = 3000


#SpiNNaker Setup
delay = dt
node_id = p.setup(timestep=dt, min_delay=delay, max_delay=delay)     
p.set_number_of_neurons_per_core(p.IF_curr_exp, 100) #  100 neurons per core


celltype = p.IF_curr_exp
 
con_move = []

port = 19600


p_labels = ["one", "two", "three", "four"]
m_labels = ["right", "left", "up", "down"]

cell_conn = p.OneToOneConnector()

px_neurons = []
for i in range(len(m_labels)):
    px_neurons.append(p.Population(1, p.SpikeSourcePoisson(rate=0), label=p_labels[i]))
#     px_neurons[i].record(["v","spikes"])

# Spike emission (from CPU to SpiNNaker)        
external_input_control = p.external_devices.SpynnakerPoissonControlConnection(poisson_labels=p_labels, local_port=port+1)
for i in range(4):
    p.external_devices.add_poisson_live_rate_control(px_neurons[i], database_notify_port_num=port+1)
    external_input_control.add_start_resume_callback(p_labels[i], send_spikes_to_sim)
    
move_neurons = []
for i in range(len(m_labels)):
    move_neurons.append(p.Population(1, celltype(**cell_params), label=m_labels[i]))
#     move_neurons[i].record(["v","spikes"])

# Spike reception (from SpiNNaker to CPU)
live_spikes_receiver = p.external_devices.SpynnakerLiveSpikesConnection(receive_labels=m_labels, local_port=port-1)
for i in range(4):
    p.external_devices.activate_live_output_for(move_neurons[i], database_notify_port_num=live_spikes_receiver.local_port)
    live_spikes_receiver.add_receive_callback(m_labels[i], receive_spikes_from_sim)

        
#################
#       #       #
#   1   #   2   #
#       #       #
#################
#       #       #
#   3   #   4   #
#       #       #
#################


con_move.append({'e_one_right': p.Projection(px_neurons[0], move_neurons[0], cell_conn,
                                            receptor_type='excitatory',
                                            synapse_type=p.StaticSynapse(weight=w, delay=delay))})

con_move.append({'e_one_down': p.Projection(px_neurons[0], move_neurons[3], cell_conn,
                                            receptor_type='excitatory',
                                            synapse_type=p.StaticSynapse(weight=w, delay=delay))})

con_move.append({'e_two_left': p.Projection(px_neurons[1], move_neurons[1], cell_conn,
                                            receptor_type='excitatory',
                                            synapse_type=p.StaticSynapse(weight=w, delay=delay))})

con_move.append({'e_two_down': p.Projection(px_neurons[1], move_neurons[3], cell_conn,
                                            receptor_type='excitatory',
                                            synapse_type=p.StaticSynapse(weight=w, delay=delay))})

con_move.append({'e_three_right': p.Projection(px_neurons[2], move_neurons[0], cell_conn,
                                            receptor_type='excitatory',
                                            synapse_type=p.StaticSynapse(weight=w, delay=delay))})

con_move.append({'e_three_up': p.Projection(px_neurons[2], move_neurons[2], cell_conn,
                                            receptor_type='excitatory',
                                            synapse_type=p.StaticSynapse(weight=w, delay=delay))})

con_move.append({'e_four_left': p.Projection(px_neurons[3], move_neurons[1], cell_conn,
                                            receptor_type='excitatory',
                                            synapse_type=p.StaticSynapse(weight=w, delay=delay))})

con_move.append({'e_four_up': p.Projection(px_neurons[3], move_neurons[2], cell_conn,
                                            receptor_type='excitatory',
                                            synapse_type=p.StaticSynapse(weight=w, delay=delay))})




con_move.append({'i_one_left': p.Projection(px_neurons[0], move_neurons[1], cell_conn,
                                            receptor_type='inhibitory',
                                            synapse_type=p.StaticSynapse(weight=w, delay=delay))})

con_move.append({'i_one_up': p.Projection(px_neurons[0], move_neurons[2], cell_conn,
                                            receptor_type='inhibitory',
                                            synapse_type=p.StaticSynapse(weight=w, delay=delay))})

con_move.append({'i_two_right': p.Projection(px_neurons[1], move_neurons[0], cell_conn,
                                            receptor_type='inhibitory',
                                            synapse_type=p.StaticSynapse(weight=w, delay=delay))})

con_move.append({'i_two_up': p.Projection(px_neurons[1], move_neurons[2], cell_conn,
                                            receptor_type='inhibitory',
                                            synapse_type=p.StaticSynapse(weight=w, delay=delay))})

con_move.append({'i_three_left': p.Projection(px_neurons[2], move_neurons[1], cell_conn,
                                            receptor_type='inhibitory',
                                            synapse_type=p.StaticSynapse(weight=w, delay=delay))})

con_move.append({'i_three_down': p.Projection(px_neurons[2], move_neurons[3], cell_conn,
                                            receptor_type='inhibitory',
                                            synapse_type=p.StaticSynapse(weight=w, delay=delay))})

con_move.append({'i_four_right': p.Projection(px_neurons[3], move_neurons[0], cell_conn,
                                            receptor_type='inhibitory',
                                            synapse_type=p.StaticSynapse(weight=w, delay=delay))})

con_move.append({'i_four_down': p.Projection(px_neurons[3], move_neurons[3], cell_conn,
                                            receptor_type='inhibitory',
                                            synapse_type=p.StaticSynapse(weight=w, delay=delay))})        


2022-03-22 08:55:18 INFO: Read cfg files: /home/juan/spinstuff/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg, /home/juan/spinstuff/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg, /home/juan/.spynnaker.cfg
2022-03-22 08:55:18 INFO: Will search these locations for binaries: /home/juan/spinstuff/lib/python3.6/site-packages/spinn_front_end_common/common_model_binaries : /home/juan/spinstuff/lib/python3.6/site-packages/spynnaker/pyNN/model_binaries
2022-03-22 08:55:18 WARNING: /home/juan/CLSpiNN/experiments/closed_loop/reports has 24 old reports that have not been closed
2022-03-22 08:55:18 INFO: Setting time scale factor to 1.
2022-03-22 08:55:18 INFO: Setting machine time step to 1000 micro-seconds.
2022-03-22 08:55:18 INFO: 0.0.0.0:19601 Waiting for message to indicate that the database is ready
2022-03-22 08:55:18 INFO: 0.0.0.0:19599 Waiting for message to indicate that the database is ready
2022-03-22 08:55:18 INFO: Receive callback <function rece

['/home/juan/spinstuff/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/juan/spinstuff/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/juan/.spynnaker.cfg']


In [5]:

# Run simulation 
p.run(nb_steps)

2022-03-22 08:55:18 INFO: Starting execution process
2022-03-22 08:55:18 INFO: Simulating for 3000 1.0ms timesteps using a hardware timestep of 1000us
2022-03-22 08:55:18 INFO: Creating transceiver for 172.16.223.29
2022-03-22 08:55:18 INFO: Working out if machine is booted
2022-03-22 08:55:22 INFO: Attempting to boot machine
2022-03-22 08:55:28 INFO: Found board with version [Version: SC&MP 3.4.1 at SpiNNaker:0:0:0 (built Thu Feb 11 15:36:44 2021)]
2022-03-22 08:55:28 INFO: Machine communication successful
2022-03-22 08:55:28 INFO: Detected a machine on IP address 172.16.223.29 which has 856 cores and 120.0 links
2022-03-22 08:55:28 INFO: Time 0:00:09.974923 taken by MachineGenerator
Preallocating resources for Live Recording
|0%                          50%                         100%|
2022-03-22 08:55:28 INFO: Time 0:00:00.005750 taken by PreAllocateResourcesForLivePacketGatherers
Preallocating resources for Extra Monitor support vertices
|0%                          50%           

Generating data specifications
|0%                          50%                         100%|
 =================================================/home/juan/spinstuff/lib/python3.6/site-packages/spynnaker/pyNN/models/spike_source/spike_source_poisson_machine_vertex.py:449: RuntimeWarning: divide by zero encountered in true_divide
  (1.0 / spikes_per_tick).astype(int), 0).astype("uint32")
2022-03-22 08:55:29 INFO: Time 0:00:00.094299 taken by SpynnakerDataSpecificationWriter
Preparing Routing Tables
|0%                          50%                         100%|
2022-03-22 08:55:29 INFO: Time 0:00:00.045822 taken by RoutingSetup
Finding binaries
|0%                          50%                         100%|
2022-03-22 08:55:29 INFO: Time 0:00:00.035498 taken by GraphBinaryGatherer
Running pair routing table compression on chip
|0%                          50%                         100%|

2022-03-22 08:55:32 INFO: Time 0:00:03.436943 taken by PairOnChipRouterCompression
Generating Router 

Updating run time
|0%                          50%                         100%|
2022-03-22 08:55:56 INFO: Time 0:00:00.042765 taken by ChipRuntimeUpdater
2022-03-22 08:55:56 INFO: *** Running simulation... *** 
Loading buffers
|0%                          50%                         100%|
2022-03-22 08:55:56 INFO: ** Awaiting for a response from an external source to state its ready for the simulation to start **
2022-03-22 08:55:56 INFO: ** Sending start / resume message to external sources to state the simulation has started or resumed. **
2022-03-22 08:55:56 INFO: ** Awaiting for a response from an external source to state its ready for the simulation to start **
2022-03-22 08:55:56 INFO: Waiting for message to indicate that the simulation has stopped or paused
2022-03-22 08:55:56 INFO: Application started; waiting 3.1s for it to stop
2022-03-22 08:55:56 INFO: Waiting for message to indicate that the simulation has stopped or paused


1 for up
1 for left
1 for up
1 for up
1 for up
1 for right
1 for up
1 for up
1 for left
1 for up
1 for up
1 for up
1 for up
1 for up
1 for up
1 for right
1 for up
1 for left
1 for up
1 for up
1 for up
1 for up
1 for up
1 for up
1 for right
1 for up
1 for right
1 for up
1 for up
1 for up
1 for right
1 for up
1 for right
1 for up
1 for right
1 for up
1 for right
1 for up
1 for left
1 for up
1 for up
1 for left


2022-03-22 08:55:59 INFO: ** Sending pause / stop message to external sources to state the simulation has been paused or stopped. **
2022-03-22 08:55:59 INFO: Time 0:00:03.218051 taken by ApplicationRunner
Extracting IOBUF from the machine
|0%                          50%                         100%|
2022-03-22 08:56:00 INFO: Time 0:00:00.049383 taken by ChipIOBufExtractor
clearing IOBUF from the machine
|0%                          50%                         100%|
2022-03-22 08:56:00 INFO: Time 0:00:00.016173 taken by ChipIOBufClearer
2022-03-22 08:56:00 INFO: No recorded data to extract
2022-03-22 08:56:00 INFO: Time 0:00:00.000709 taken by BufferExtractor
2022-03-22 08:56:00 INFO: Time 0:00:00.000097 taken by FinaliseTimingData
Getting provenance data
|0%                          50%                         100%|
2022-03-22 08:56:00 INFO: Time 0:00:00.043309 taken by PlacementsProvenanceGatherer
2022-03-22 08:56:00 INFO: Time 0:00:00.000932 taken by RedundantPacketCountReport
Gett

3000.0

In [6]:

p.end()